In [3]:
## imports 
import pandas as pd
import numpy as np
import yaml
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# comment these out if you don't have plotnine--not essential here/only used once
import matplotlib.pyplot as plt
# import plotnine
# from plotnine import *

## way to connect to mysql 
## if you need to install
## uncomment this line:
#! pip install mysql-connector-python
import mysql.connector

## function to feed path name to load
## credentials
def load_creds(path: str):
    with open(path, 'r') as stream:
        try:
            creds = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    return(creds)

pd.options.display.max_rows = 999
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Preliminary: define connection and read sample of data

In [5]:
## read in creds; change the path name if stored
## elsewhere
creds = load_creds("../activities/09_db_cred.yaml")

In [6]:
creds


{'practice_database': {'user': 'qss20',
  'password': 'xtPdDZHnxKMFSD5RyP',
  'port': 3306,
  'database': 'sentencing',
  'host': 'qss20-7570-db.c.dartmouth.edu'}}

In [7]:
creds['practice_database']['database']

'sentencing'

In [11]:
cnx = mysql.connector.connect(
    user=creds['practice_database']['user'],
    password = creds['practice_database']['password'],
    port = creds['practice_database']['port'],
    database = creds['practice_database']['database'],
    host = creds['practice_database']['host']
)
cursor = cnx.cursor(buffered=True)
cnx

In [10]:
q = """
SELECT
    * 
FROM caseinit
LIMIT 5
"""
pd.read_sql_query(q, cnx)

,CASE_ID,CASE_PARTICIPANT_ID,RECEIVED_DATE,OFFENSE_CATEGORY,PRIMARY_CHARGE_FLAG,CHARGE_ID,CHARGE_VERSION_ID,DISPOSITION_CHARGED_OFFENSE_TITLE,CHARGE_COUNT,DISPOSITION_DATE,...,INCIDENT_END_DATE,LAW_ENFORCEMENT_AGENCY,LAW_ENFORCEMENT_UNIT,ARREST_DATE,FELONY_REVIEW_DATE,FELONY_REVIEW_RESULT,ARRAIGNMENT_DATE,UPDATED_OFFENSE_CATEGORY,is_in_diversion,in_chicago
0,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242195814523,155656315869,FIRST DEGREE MURDER,2,12/17/2014 12:00:00 AM,...,,CHICAGO POLICE DEPT,,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion,False,None
1,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242198287388,131513547452,HOME INVASION,14,12/17/2014 12:00:00 AM,...,,CHICAGO POLICE DEPT,,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion,False,None
2,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242351605056,176626576281,FIRST DEGREE MURDER,4,12/17/2014 12:00:00 AM,...,,CHICAGO POLICE DEPT,,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion,False,None
3,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242352841488,176617824190,FIRST DEGREE MURDER,5,12/17/2014 12:00:00 AM,...,,CHICAGO POLICE DEPT,,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion,False,None
4,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242356550787,131238606761,HOME INVASION,13,12/17/2014 12:00:00 AM,...,,CHICAGO POLICE DEPT,,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion,False,None


# Activity 1

1. Create a new column -- `in_chicago` when pulling from the `caseinit` table that takes on the value of "YES" if INCIDENT_CITY = Chicago; "NO" otherwise (which represents incidents in Cook County suburbs outside the city limits);  and pull the table. Use `crosstabs` to confirm that this worked
2. Repeat step 1 but also filter out blank strings (`INCIDENT_CITY` == "")
3. Use `where` to row filter to initiations in Chicago and use group by to find the count of cases diverted and not diverted (`is_in_diversion`); pull the table with those counts
4. Modify the query in step 3 to find the proportion of cases in chicago diverted (hint you made need to use case when in a subquery)
5. Modify the query in step 4 to find the proportion of cases in chicago versus cases not in chicago sent to diversion 


In [29]:
# your code here 1
q = """
SELECT
    INCIDENT_CITY,
    CASE 
        WHEN INCIDENT_CITY = 'Chicago' THEN 'YES'
        ELSE 'NO'
    END AS in_chicago1
FROM caseinit;
"""
incidents = pd.read_sql_query(q, cnx)

# crosstab
crosstab = pd.crosstab(incidents['INCIDENT_CITY'], incidents['in_chicago1'])
print(crosstab)


in_chicago1               NO     YES
INCIDENT_CITY                       
                       20557       0
Addison                    2       0
Albers                     1       0
Algonquin                  3       0
Alsip                    568       0
Antioch                    2       0
Arlington Heights       1291       0
Aurora                     3       0
Barrington               136       0
Barrington Hills          16       0
Bartlett                 241       0
Bedford Park             335       0
Beecher                    1       0
Bellwood                 848       0
Bensenville                3       0
Berkeley                 160       0
Berwyn                  2152       0
Bloomingdale               1       0
Bloomington                1       0
Blue Island             1192       0
Bluff Springs              1       0
Bolingbrook                5       0
Bourbonnais                1       0
Braidwood                  1       0
Bridgeview               704       0
B

In [18]:
# your code here 2
q = """
SELECT
    INCIDENT_CITY,
    CASE 
        WHEN INCIDENT_CITY = 'Chicago' THEN 'YES'
        ELSE 'NO'
    END AS in_chicago1
FROM caseinit;
"""
pd.read_sql_query(q, cnx)

,INCIDENT_CITY,in_chicago,incident_count
0,Addison,NO,2
1,Albers,NO,1
2,Algonquin,NO,3
3,Alsip,NO,568
4,Antioch,NO,2
5,Arlington Heights,NO,1291
6,Aurora,NO,3
7,Barrington,NO,136
8,Barrington Hills,NO,16
9,Bartlett,NO,241


In [ ]:
# your code here 3

In [ ]:
# your code here 4

In [ ]:
# your code here 5

# Activity 2 

1. Use the following crosswalk and the `CASE` variable in the `divert` table to create a new variable `DIVERSION_PROGRAM_TEXT` that spells out the diversion programs
    - DC: Drug Court

    - DDPP: Drug Deferred Prosecution

    - DS: Drug School

    - RJCC: Restorative Justice

    - MHC: Mental Health Court

    - VC: Veteran Court

2. Build on the query from step 1 to filter to Narcotics as the `UPDATED_OFFENSE_CATEGORY` and Black or White defendants (based on race in the diversions table) (hint: you'll need to join with the caseinit table based on case_id and case_participant_id, you can do a inner join to keep only those diverted). Select the case_id, case_participant_id, case, race, and diversion_program_text columns

In [ ]:
# your code here 1

In [ ]:
# your code here 2